In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import itertools
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn import model_selection
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import metrics

# Classifiers
from sklearn.svm import NuSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingCVClassifier # <- Here is our boy

# Used to ignore warnings generated from StackingCVClassifier
import warnings
warnings.simplefilter('ignore')

In [8]:
review=pd.read_csv('Review_project_sentiment.csv')
review.head(3)

,Unnamed: 0,review_id,user_id,business_id,stars,useful,funny,cool,text,date,useful_level,language,text_clean,read_ease,polarity,subjectivity,Adj_ratio
0,1073206,JdReKgETiiJEDmshrO4TLw,pyarmAnR-i-qookQamqRTA,V2GOReqPvr8qpCC7sWfoTw,1.0,17,1,0,Just to let this car company that people DO re...,2014-03-06 12:38:52,moderate useful,en,let car company people read yelp check review ...,85.49,-0.500000,1.000000,0.120000
1,6229216,zL4se_Ixdcl8kvTOHCS3rg,s16-BUo-orUsELvMu5ocKg,VH0Ib9S3E-dxbQdQC4rffg,1.0,15,6,4,Mistral was the worst dining experience I have...,2010-07-22 18:08:01,moderate useful,en,mistral worst dining experience ever life bad ...,79.19,0.029864,0.502499,0.225888
2,7607157,wlLQ42QMHiQMJISCTUVX5A,aQA2eIdEC3_ZDHwUregu8A,wygOtij7aEb7dTdTFFnOcw,5.0,0,0,0,Took a VIP tour with DeMon of the Atlanta Merc...,2020-02-04 16:05:20,not useful,en,took vip tour demon atlanta mercedes benz stad...,-32.40,0.422500,0.696111,0.218750


In [13]:
#encoder
review1=review
review1.loc[review1['useful_level']=="useful",'useful_level']=2
review1.loc[review1['useful_level']=="moderate useful",'useful_level']=1
review1.loc[review1['useful_level']=="not useful",'useful_level']=0

#encoder1=LabelEncoder()
#review1['useful_level'] = encoder1.fit_transform(review1['useful_level'].values)
review1

,Unnamed: 0,review_id,user_id,business_id,stars,useful,funny,cool,text,date,useful_level,language,text_clean,read_ease,polarity,subjectivity,Adj_ratio
0,1073206,JdReKgETiiJEDmshrO4TLw,pyarmAnR-i-qookQamqRTA,V2GOReqPvr8qpCC7sWfoTw,1.0,17,1,0,Just to let this car company that people DO re...,2014-03-06 12:38:52,1,en,let car company people read yelp check review ...,85.49,-0.500000,1.000000,0.120000
1,6229216,zL4se_Ixdcl8kvTOHCS3rg,s16-BUo-orUsELvMu5ocKg,VH0Ib9S3E-dxbQdQC4rffg,1.0,15,6,4,Mistral was the worst dining experience I have...,2010-07-22 18:08:01,1,en,mistral worst dining experience ever life bad ...,79.19,0.029864,0.502499,0.225888
2,7607157,wlLQ42QMHiQMJISCTUVX5A,aQA2eIdEC3_ZDHwUregu8A,wygOtij7aEb7dTdTFFnOcw,5.0,0,0,0,Took a VIP tour with DeMon of the Atlanta Merc...,2020-02-04 16:05:20,0,en,took vip tour demon atlanta mercedes benz stad...,-32.40,0.422500,0.696111,0.218750
3,141509,OMw52-Eo-BYFHdUldLMc5Q,mmr-P0i__uQJHGXXeVjXxg,0Ng-wDNyA-96uaJZP8EKZg,4.0,40,0,0,When I first came here for the first time last...,2015-03-09 22:42:06,2,en,first came first time last year best friend ap...,83.96,0.168452,0.419841,0.168831
4,7962637,RWq78T9FIyauVHaHj7yl9g,CyCI71vLDa923zOMeDDA5w,WUfZXjTtBUbezJE7LnWABw,3.0,0,0,0,"I only got the take-out, so I cannot speak of ...",2013-03-22 20:47:40,0,en,got takeout cannot speak service food good wou...,51.86,0.320000,0.780000,0.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55182,6543262,LGMsIuesuDdK3lH3dM-dWA,3PYDOk5Us28fuHZAUEZq5g,ejzWpdWRZu4JM9tsZgZVEA,5.0,11,0,4,Wow. I had some gold rings/necklaces to sell a...,2014-06-02 21:08:24,1,en,wow gold ringsnecklaces sell took gold buyer c...,80.92,0.164017,0.426303,0.150943
55183,5441853,K7oX_4ZXoHxFAJV8IBDC0w,vpoRmKwfFDW3hj5baBNdiA,ggqr9BEklylICKl770GDlQ,1.0,1,1,1,Use to love to go here at least once a month. ...,2017-06-13 07:59:41,0,en,use love go least month last couple time food ...,102.61,-0.045833,0.602778,0.176471
55184,1415430,ojJGvcMZObHdHrvvm__Qlw,yROiz_5pQose97qyETplzQ,ICtCTow-58Ie24hOC1eG3w,5.0,0,0,0,"They got tripe, they got beef tongue, they got...",2017-06-05 23:49:32,0,en,got tripe got beef tongue got cheapest shrimp ...,91.11,0.263803,0.544530,0.276923
55185,5602623,sFQ8waEtUsguHfAgbxhGJA,AkBZo05E0KApriZqxXgfRQ,x245eNNdQ6i4FbZsDeCdJw,5.0,13,28,15,"Great bar, or greatest bar? \n\nMaybe it's my ...",2011-11-21 22:49:25,1,en,great bar greatest bar maybe blue collar root ...,73.71,0.191353,0.491893,0.185841


In [10]:
review1.to_csv('review_sentiment_encoded.csv')

In [5]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [24]:
y = review1["useful_level"]

In [25]:
X = review1.iloc[: , -4:]

In [40]:
y

0        1
1        1
2        0
3        2
4        0
        ..
55182    1
55183    0
55184    0
55185    1
55186    1
Name: useful_level, Length: 55187, dtype: object

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20,
                                                    random_state = 1000)

In [47]:
y_test

16501    0
48589    1
49157    1
39598    1
50624    1
        ..
40386    2
24725    0
26727    1
53068    0
3217     0
Name: useful_level, Length: 11038, dtype: object

In [43]:
classifier1 = SVC(C = 50, degree = 1, gamma = "auto", kernel = "rbf", probability = True)

# Initializing Multi-layer perceptron  classifier
classifier2 = MLPClassifier(activation = "relu", alpha = 0.1, hidden_layer_sizes = (10,10,10),
                            learning_rate = "constant", max_iter = 2000, random_state = 1000)

# Initialing Nu Support Vector classifier
classifier3 = NuSVC(degree = 1, kernel = "rbf", nu = 0.25, probability = True)

# Initializing Random Forest classifier
classifier4 = RandomForestClassifier(n_estimators = 500, criterion = "gini", max_depth = 10,
                                     max_features = "auto", min_samples_leaf = 0.005,
                                     min_samples_split = 0.005, n_jobs = -1, random_state = 1000)

In [44]:
sclf = StackingCVClassifier(classifiers = [classifier1, classifier2, classifier3, classifier4],
                            shuffle = False,
                            use_probas = True,
                            cv = 5,
                            meta_classifier = SVC(probability = True))

In [45]:
classifiers = {"SVC": classifier1,
               "MLP": classifier2,
               "NuSVC": classifier3,
               "RF": classifier4,
               "Stack": sclf}

In [ ]:
for key in classifiers:
    # Get classifier
    classifier = classifiers[key]
    
    # Fit classifier
    classifier.fit(X_train, y_train.astype(int))
        
    # Save fitted classifier
    classifiers[key] = classifier

In [ ]:
results = pd.DataFrame()
for key in classifiers:
    # Make prediction on test set
    y_pred = classifiers[key].predict_proba(X_test)[:,1]
    
    # Save results in pandas dataframe object
    results[f"{key}"] = y_pred

# Add the test set to the results object
results["Target"] = y_test

In [ ]:
sns.set(font_scale = 1)
sns.set_style({"axes.facecolor": "1.0", "axes.edgecolor": "0.85", "grid.color": "0.85",
               "grid.linestyle": "-", 'axes.labelcolor': '0.4', "xtick.color": "0.4",
               'ytick.color': '0.4'})

# Plot
f, ax = plt.subplots(figsize=(13, 4), nrows=1, ncols = 5)

for key, counter in zip(classifiers, range(5)):
    # Get predictions
    y_pred = results[key]
    
    # Get AUC
    auc = metrics.roc_auc_score(y_test, y_pred)
    textstr = f"AUC: {auc:.3f}"

    # Plot false distribution
    false_pred = results[results["Target"] == 0]
    sns.distplot(false_pred[key], hist=True, kde=False, 
                 bins=int(25), color = 'red',
                 hist_kws={'edgecolor':'black'}, ax = ax[counter])
    
    # Plot true distribution
    true_pred = results[results["Target"] == 1]
    sns.distplot(results[key], hist=True, kde=False, 
                 bins=int(25), color = 'green',
                 hist_kws={'edgecolor':'black'}, ax = ax[counter])
    
    
    # These are matplotlib.patch.Patch properties
    props = dict(boxstyle='round', facecolor='white', alpha=0.5)
    
    # Place a text box in upper left in axes coords
    ax[counter].text(0.05, 0.95, textstr, transform=ax[counter].transAxes, fontsize=14,
                    verticalalignment = "top", bbox=props)
    
    # Set axis limits and labels
    ax[counter].set_title(f"{key} Distribution")
    ax[counter].set_xlim(0,1)
    ax[counter].set_xlabel("Probability")

# Tight layout
plt.tight_layout()

# Save Figure
plt.savefig("Probability Distribution for each Classifier.png", dpi = 1080)